In [6]:
import json
import pandas as pd
import gf_utils
from gf_utils.stc_data import get_stc_data
from pathlib import Path
from datetime import datetime
from collections import defaultdict

log_dir = Path('logs/equip')
output_file = Path('equip.csv')
full_record_df = pd.read_csv(output_file)
game_data = get_stc_data(Path(gf_utils.__file__).resolve().parent/'../../data/ch/formatted/json')

In [5]:
from rich.table import Table
from rich.console import Console
from rich.style import NULL_STYLE
from datetime import datetime
td = int(datetime(2022,11,18,10).timestamp())
console = Console(force_terminal=True,force_jupyter=False,no_color=True,highlight=False)
for formula in game_data['recommended_formula'].values():
    # print(formula)
    if formula['develop_type']!=3:
        continue
    mp,ammo,mre,part = formula['mp'],formula['ammo'],formula['mre'],formula['part']
    common_dev_df = full_record_df.query('mp==@mp and ammo==@ammo and mre==@mre and part==@part and item_num==1 and trust==True and dev_time>@td')['equip_id'].value_counts().to_frame('count')
    common_dev_df['chance'] = common_dev_df['count'].apply(lambda x: round(x/common_dev_df['count'].sum()*100,3))
    special_dev_df = full_record_df.query('mp==@mp and ammo==@ammo and mre==@mre and part==@part and item_num==0 and trust==True and dev_time>@td')['equip_id'].value_counts().to_frame('count')
    special_dev_df['chance'] = special_dev_df['count'].apply(lambda x: round(x/special_dev_df['count'].sum()*100,3))
    dev_df = common_dev_df.join(special_dev_df,rsuffix='_special',how='outer').fillna(0).reset_index()
    dev_df['count_special'] = dev_df['count_special'].astype(int)
    for key in ['name','rank']:
        dev_df[key] = dev_df['index'].apply(lambda x: game_data['equip'][x][key])
    footer = f"Count: common {dev_df['count'].sum()}, special {dev_df['count_special'].sum()}\n"
    dev_df = dev_df.query('rank==5').sort_values('chance',ascending=False)
    footer += f"Rank5: common {dev_df['chance'].sum():.3f}, special {dev_df['chance_special'].sum():.3f}"
    table = Table('           name','common','special',header_style=NULL_STYLE, title_style=NULL_STYLE, style=NULL_STYLE, caption=footer, caption_justify='left', caption_style=NULL_STYLE)
    for col in table.columns:
        col.justify='right'
    table.title = f"Formula: {mp} {ammo} {mre} {part}"
    for idx, record in dev_df.iterrows():
        table.add_row(record['name'],*[f'{record[k]:.3f}' for k in ['chance','chance_special']])
    console.print(table)
    console.print()

        Formula: 140 10 110 10        
┌─────────────────┬────────┬─────────┐
│            name │ common │ special │
├─────────────────┼────────┼─────────┤
│     VFL 6-24X56 │  1.984 │   1.606 │
│         EOT 518 │  1.984 │   0.803 │
│        ITI MARS │  0.794 │   0.201 │
│       AC4消音器 │  0.397 │   0.000 │
│      SF武器灯改 │  0.397 │   0.000 │
└─────────────────┴────────┴─────────┘
Count: common 252, special 498        
Rank5: common 5.556, special 2.610    

        Formula: 140 10 110 10        
┌─────────────────┬────────┬─────────┐
│            name │ common │ special │
├─────────────────┼────────┼─────────┤
│     VFL 6-24X56 │  1.984 │   1.606 │
│         EOT 518 │  1.984 │   0.803 │
│        ITI MARS │  0.794 │   0.201 │
│       AC4消音器 │  0.397 │   0.000 │
│      SF武器灯改 │  0.397 │   0.000 │
└─────────────────┴────────┴─────────┘
Count: common 252, special 498        
Rank5: common 5.556, special 2.610    

        Formula: 140 10 110 10        
┌─────────────────┬────────┬─────────

In [8]:
full_record_df.query('user_id==753332')

,id,user_id,dev_type,build_slot,dev_uname,dev_lv,equip_id,mp,ammo,mre,part,input_level,core,item_num,dev_time,fairy_id,passive_skill,quality_lv,file,trust
3531,28843855,753332,0,1,凌绰约,254,14,70,10,150,30,0,0,1,1666966420,0,0,0,221028-222422.json,True
3532,28843856,753332,0,3,凌绰约,254,9,70,10,150,30,0,0,1,1666966420,0,0,0,221028-222422.json,True
3533,28843857,753332,0,1,凌绰约,254,222,70,10,150,30,0,0,1,1666966423,0,0,0,221028-222422.json,True
3534,28843858,753332,0,3,凌绰约,254,13,70,10,150,30,0,0,1,1666966423,0,0,0,221028-222422.json,True
3538,28843862,753332,0,1,凌绰约,254,232,270,40,270,190,0,0,1,1666966538,0,0,0,221028-222422.json,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113749,28986738,753332,0,3,凌绰约,260,305,70,10,150,30,0,0,1,1669865222,0,0,0,221201-113344.json,True
113755,28986746,753332,0,1,凌绰约,260,6,70,10,150,30,0,0,1,1669865459,0,0,0,221201-114438.json,True
113756,28986747,753332,0,3,凌绰约,260,1,70,10,150,30,0,0,1,1669865525,0,0,0,221201-114438.json,True
113804,28986803,753332,0,1,凌绰约,260,88,70,10,150,30,0,0,1,1669867601,0,0,0,221201-121213.json,True


In [9]:
3872/113808

0.0340222128497118

In [7]:
[f'{record[k]:.3f}' for k in ['name','chance','chance_special']]

['Mk2', '0.031', '0.0']

In [15]:
f'{0.0:.3}'

'0.0'